In [17]:
from urllib2 import urlopen
from urllib import urlencode
from urllib import quote_plus
# import urllib.parse
import json
import pprint
import os

class CoreApiRequestor:

    def __init__(self, endpoint, api_key):
        self.endpoint = endpoint
        self.api_key = api_key
        #defaults
        self.pagesize = 100
        self.page = 1

    def parse_response(self, decoded):
        res = []
        for item in decoded['data']:
            doi = None
            if 'identifiers' in item:
                for identifier in item['identifiers']:
                    if identifier and identifier.startswith('doi:'):
                        doi = identifier
                        break
            res.append([item['title'], doi])
        return res

    def request_url(self, url):
        print(url)
        response = urlopen(url)
        html = response.read()
        return html

    def get_method_query_request_url(self,method,query,fullText,page):
        if (fullText):
            fullText = 'true'
        else:
            fullText = 'false'
        params = {
            'apiKey':self.api_key,
            'page':page,
            'pageSize':self.pagesize,
            'fulltext':fullText
        }
        return self.endpoint + method + '/' + quote_plus(query) + '?' + urlencode(params)

    def get_up_to_20_pages_of_query(self,method,query,fulltext):
        url = self.get_method_query_request_url(method,query,fulltext,1)
        all_articles=[]
        resp = self.request_url(url)
        result = json.loads(resp.decode('utf-8'))
        all_articles.append(result)
        if (result['totalHits']>100):
            numOfPages = int(result['totalHits']/self.pagesize)  #rounds down
            if (numOfPages>20):
                numOfPages=20
            for i in range(2,numOfPages):
                url = self.get_method_query_request_url(method,query,False,i)
                print(url)
                resp =self.request_url(url)
                all_articles.append(json.loads(resp.decode('utf-8')))
        return all_articles
    
    def get_repository_articles_fulltext_as_dict(self,repository_id):
        return ""
    
    def get_search_repository_request_url(self,repoQuery,page=1,pageSize=10):
        params={
            'apiKey':self.api_key,
            'page':page,
            'pageSize':pageSize
        }
        return self.endpoint + "/repositories/search/"+quote_plus(repoQuery)+'?'+urlencode(params)
    
    def search_repository_ids_by_name(self,repoName):
        discoverRepoUrl = self.get_search_repository_request_url(repoName)
        resp = self.request_url(discoverRepoUrl)
        result = json.loads(resp.decode('utf-8'))
        for item in result['data']:
            if 'name' in item:
                name = item['name']
            if 'id' in item:
                id = item['id']
            repos[id]=name
        return repos
    
    def get_count_articles_of_repository_url(self,repoId,withFullText):
        ft = 'false'
        if(withFullText):
            ft = 'true'
                
        params = {
            'apiKey':self.api_key,
            'fulltext':ft
        }    
        return self.endpoint + "/articles/search/repositories.id:"+str(repoId)+'?'+urlencode(params)
    
    def count_articles_of_repository(self,repoId,withFullText=False):
        countArticlesOfRepoUrl = self.get_count_articles_of_repository_url(repoId,withFullText)
        print(countArticlesOfRepoUrl)
        resp = self.request_url(countArticlesOfRepoUrl)
        result = json.loads(resp.decode('utf-8'))
        return result['totalHits']
    
    def get_url_of_download_articles_of_repository(self,repoId,fullText,page,pageSize):
        params={
            'apiKey':self.api_key,
            'page':page,
            'pageSize':pageSize,
            'fulltext':fullText
               }
        return self.endpoint + "/articles/search/repositories.id:"+str(repoId)+'?'+urlencode(params)
    
    def download_articles_of_repository(self, repoId, fulltext=True, page=1, pageSize=100):
        url = self.get_url_of_download_articles_of_repository(repoId,fulltext,page,pageSize)
        all_articles=[]
        resp = self.request_url(url)
        result = json.loads(resp.decode('utf-8'))
        if (result['totalHits']>100):
            numOfPages = int(result['totalHits']/self.pagesize)  #rounds down
            if (numOfPages>5):
                numOfPages=5
            for i in range(2,numOfPages):
                url = self.get_url_of_download_articles_of_repository(repoId,fulltext,page,pageSize)
                print(url)
                resp =self.request_url(url)
                all_articles.append(json.loads(resp.decode('utf-8')))
        return all_articles

In [18]:
'''
Initialise parameters
'''
# init 
endpoint = 'https://core.ac.uk/api-v2'

'''
********************************************
Add your own api key below
'''
api_key =""
# or get it from a config file
file = open("api_key.secret","r") 
api_key=file.read()
api_key="20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j"
api_key.strip()
'''
********************************************
'''
'''
Create your api object
'''
api = CoreApiRequestor(endpoint,api_key)

In [22]:
import pickle
def cache_repository_articles(repoId,numPages=10):
    '''
    Use page size 50 
    -
    store as pickle every 200 articles (4 pages)
    in the end you will have in the /data foledr files:
    all_articles_on1_1.pkl
    all_articles_on1_2.pkl
    all_articles_on1_3.pkl
    ...

    '''

    all_articles=[]
    for i in range(1,numPages*4):
        if not os.path.exists('./data/all_articles_on'+str(repoId)+'_'+str(i/4)+'.pkl'):
            url = api.get_url_of_download_articles_of_repository(repoId,True,i,50)
            response = urlopen(url)
            resp = response.read()
            result = json.loads(resp.decode('utf-8'))
            all_articles.append(result)
            if (i%4==0):
                pickle.dump(all_articles,open('data/all_articles_on'+str(repoId)+'_'+str(i/4)+'.pkl','wb'),pickle.HIGHEST_PROTOCOL)
                all_articles=[]
                
def cache_all_repository_articles(repoId):
    '''
    Same as cache_repository_articles but we download all of repository's articles it and not just 40 pages
    '''
    all_articles=[]
    repo_count=api.count_articles_of_repository(repoId,True)
    numOfPages=repo_count/50
    for i in range(1,numOfPages):
        if not os.path.exists('./data/repo_'+str(repoId)+'_full_download.pkl'):
            url = api.get_url_of_download_articles_of_repository(repoId,True,i,50)
            response = urlopen(url)
            resp = response.read()
            result = json.loads(resp.decode('utf-8'))
            all_articles.append(result)
            
    pickle.dump(all_articles,open('./data/repo_'+str(repoId)+'_full_download.pkl','wb'),pickle.HIGHEST_PROTOCOL)


In [20]:
# 1-> aberdeen
# 86-> Open university
# 39 -> edinburgh 
# 136 -> warwick
# 48 -> glascow
# repos_to_cache=[1,86,39,136]
repos_to_cache={
    1:'abdn.ac.uk',
    86:'open.ac.uk',
    39:'ed.ac.uk',
    136:'warwick.ac.uk',
    48:'gla.ac.uk',
    63:'le.ac.uk',
    121:'birmingham.ac.uk',
    80:'nottingham.ac.uk'
}

In [ ]:
'''
This should be enough to run almost all parts of this demo
'''
for repo in repos_to_cache:
    cache_repository_articles(repo,10)

In [7]:
api.get_url_of_download_articles_of_repository(1,True,1,10)

'https://core.ac.uk/api-v2/articles/search/repositories.id:1?fulltext=True&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j&page=1&pageSize=10'

In [7]:
api.get_count_articles_of_repository_url(1,True)

'https://core.ac.uk/api-v2/articles/search/repositories.id:1?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j'

In [12]:
api.count_articles_of_repository(48,True)

https://core.ac.uk/api-v2/articles/search/repositories.id:48?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j
https://core.ac.uk/api-v2/articles/search/repositories.id:48?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j


5626

In [23]:
'''
Run this if you want to create a full cache.
WARNING! this would take a LOT of time
'''
for repo in repos_to_cache:
    cache_all_repository_articles(repo)

https://core.ac.uk/api-v2/articles/search/repositories.id:48?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j
https://core.ac.uk/api-v2/articles/search/repositories.id:48?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j
https://core.ac.uk/api-v2/articles/search/repositories.id:1?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j
https://core.ac.uk/api-v2/articles/search/repositories.id:1?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j
https://core.ac.uk/api-v2/articles/search/repositories.id:86?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j
https://core.ac.uk/api-v2/articles/search/repositories.id:86?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j
https://core.ac.uk/api-v2/articles/search/repositories.id:39?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j
https://core.ac.uk/api-v2/articles/search/repositories.id:39?fulltext=true&apiKey=20hIsS1F5j4D2C2iXrg4Wxf7VTp4Xt1j
https://core.ac.uk/api-v2/articles/search/repositories.id:136?fulltext=true&apiKey